In [39]:
import os

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from docx import Document as Doc
import numpy as np
from transformers import BertModel, BertTokenizer
import torch
import json

In [40]:
#pip install python-docx

In [41]:
# Путь к файлу
infile = 'labar3new.docx'

# Открываем документ
doc = Doc(infile)

# Инициализация токенизатора и модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Сбор текста, таблиц и списков
all_text = []
structured_tables = []  # Для таблиц
structured_lists = []   # Для списков
list_checker = True     # Если есть пробел делаем False

def is_list_paragraph(paragraph):
    """
    Проверяет, является ли параграф частью списка (на основе стилей).
    """
    style_name = paragraph.style.name.lower()
    return "list" in style_name or "bullet" in style_name or "number" in style_name

current_list = []  # Временное хранилище для текущего списка

for paragraph in doc.paragraphs:
    text = paragraph.text.strip()
    
    # Проверяем, является ли параграф списком
    if is_list_paragraph(paragraph):
        if text:  # Добавляем элемент в текущий список
            current_list.append(text)
    else:
        # Если список закончился, сохраняем его
        if current_list:
            structured_lists.append(current_list)
            all_text.extend(current_list)  # Добавляем текст списка в общий массив
            current_list = []
        # Обрабатываем обычный параграф
        if text:
            all_text.append(text)

# Добавляем последний список, если он остался
if current_list:
    structured_lists.append(current_list)
    all_text.extend(current_list)

# Обработка таблиц
for table in doc.tables:
    table_data = []
    for row in table.rows:
        row_data = [cell.text.strip() for cell in row.cells]
        table_data.append(row_data)
    structured_tables.append({"data": table_data})

    # Конвертируем строки таблиц в текст
    table_text = [" | ".join(row) for row in table_data]
    all_text.extend(table_text)

# Объединяем текст для токенизации
combined_text = "\n".join(all_text)

# Токенизация текста
inputs = tokenizer(combined_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

# Получение эмбеддингов из BERT
with torch.no_grad():
    outputs = model(**inputs)

# Вывод скрытых состояний [CLS]-токена
cls_embedding = outputs.last_hidden_state[:, 0, :].numpy()

# Сохранение списков и таблиц в JSON
output_data = {
    "lists": structured_lists,
    "tables": structured_tables
}

with open("document_structure.json", "w", encoding="utf-8") as f:
    json.dump(output_data, f, ensure_ascii=False, indent=4)

'''
print("Эмбеддинги текста из документа:")
print(cls_embedding)

print("Структура документа сохранена в 'document_structure.json'")
'''

'\nprint("Эмбеддинги текста из документа:")\nprint(cls_embedding)\n\nprint("Структура документа сохранена в \'document_structure.json\'")\n'

In [42]:
print(structured_tables)

[{'data': [[''], ['МИНОБРНАУКИ РОССИИ'], ['Федеральное государственное бюджетное образовательное учреждение\nвысшего образования\n«МИРЭА – Российский технологический университет»\nРТУ МИРЭА']]}, {'data': [['№', '1', '2'], ['', '1', '1'], ['', '1', '3'], ['(абс)', '0.008495', '0.0009265'], ['(отн) %', '0.8495', '0.03088']]}, {'data': [['№', '1', '2'], ['', '0', '2'], ['', '1', '1'], ['(абс)', '-0.00003889', '-0.5724'], ['(отн) %', '-0.003889 %', '-57.24 %']]}, {'data': [['№', '1', '2'], ['', '0', '0'], ['', '0.3*sin(2*t)+2', '0.5*sin(2*t)+2']]}, {'data': [['№', '1', '2', '3', '4'], ['', '0', '0', '0', '0'], ['', '1', '1', '1', '1'], ['dt', '0.0001', '0.001', '0.005', '0.01'], ['(абс)', '-0.00003889', '0.001029', '19.4', '1096'], ['(отн) %', '-0.003889', '0.1029', '1940', '109600']]}]


In [43]:
from uuid import uuid4

from langchain_core.documents import Document

references = [combined_text]

documents = [Document(
    page_content=ref,
    metadata={"source": "tweet"}
    )
    for ref in references]

uuids = [str(uuid4()) for _ in range(len(documents))]

db = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text",show_progress=True))
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

# Set up the local model:
local_model = "llama3"
llm = ChatOllama(model=local_model, num_predict=400,
                 stop=["<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"])

# Set up the RAG chain:
prompt_template = """
<|start_header_id|>user<|end_header_id|>
Responds user questions taking into accoun the given context, give a precise and short answer without referring to this part of prompt.
Question: {question}
Context: {context}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Querying the LLM (oviously to test here you must ask a relevant question of your data)
question = "Кто автор работы?"
candidate = [rag_chain.invoke(question)]
print(question)
print(candidate[0])

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it]


Кто автор работы?
Автор работы - студент группы КРБО-01-19 Ткачев Н. А.


In [44]:
print(db)

In [45]:
#pip install faiss-cpu

In [46]:
from transformers import BertTokenizer, BertModel
from bert_score import BERTScorer
# Example texts
reference = [combined_text]
candidate = [rag_chain.invoke(question)]
# BERTScore calculation
scorer = BERTScorer(model_type='bert-base-uncased')
P, R, F1 = scorer.score(candidate, reference)
print(f"BERTScore Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it]


BERTScore Precision: 0.8156, Recall: 0.5690, F1: 0.6703


In [47]:
# Define the reference and candidate sentences
#reference = ["The cat sat on the mat."]
#candidate = ["The cat is on the mat!"]

# Compute the BERTScore
P, R, F1 = scorer.score(candidate, reference)

# Print the scores
print("Precision: {:.2f}, Recall: {:.2f}, F1: {:.2f}".format(P.item(), R.item(), F1.item()))

Precision: 0.82, Recall: 0.57, F1: 0.67
